In [1]:
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import pickle
from multiprocessing import Pool
from tqdm import tqdm
import time
import pickle as pk

### embed2ndarray 
- 赛方提供了txt格式的词向量和字向量，这里把embedding矩阵转成 np.ndarray 形式，分别保存为 data/word_embedding.npy 和 data/char_embedding.npy
- 用 pd.Series 保存词(字)对应 embedding 中的行号(id),存储在 data/sr_word2id.pkl 和 data/sr_char2id.pkl 中。

In [22]:
r = open('../raw_data/word_embedding.txt',"r")
lines = r.readlines()
for i in range(1,len(lines)):
    if( i<5 ):
        print(lines[i-1])

411720 256

</s> 0.001329 0.000907 -0.000973 -0.000106 -0.000187 0.001701 0.000681 0.000016 -0.000586 0.001416 -0.001190 0.001639 0.001556 0.000490 -0.000055 -0.001887 0.001778 -0.000790 0.001184 -0.001943 0.001400 -0.000459 -0.001033 0.001361 0.001861 -0.000285 0.000075 -0.000082 -0.000975 0.000526 -0.001432 -0.001003 -0.000174 0.001300 0.001136 -0.001540 0.001949 -0.000857 0.000424 -0.001815 -0.001502 -0.001217 0.001022 0.000098 -0.000133 0.000088 0.000013 0.000757 0.000911 0.001311 0.000802 -0.000740 0.000257 0.000405 0.001527 -0.000249 -0.001041 -0.000721 0.001283 0.000989 -0.001644 0.001727 0.000604 0.001490 -0.001362 -0.000926 0.000933 -0.000995 -0.000811 0.000411 0.001811 -0.001154 0.000234 0.000870 0.000989 0.000793 0.001250 0.000353 -0.001442 -0.001589 0.001456 0.000215 -0.001623 -0.001354 0.000156 0.000920 0.000671 0.000430 -0.000819 0.000496 0.001884 -0.000313 -0.000880 0.000770 0.000297 0.001832 -0.000638 -0.001606 0.001753 0.001320 -0.000870 0.000329 -0.001782 0.001442 0.0

In [21]:
# <PAD> <UNK> </s> w11', 'w54', ...
word_embedding = np.load('../data/word_embedding.npy')
print(word_embedding[0:5])
print(len(word_embedding))
print(len(word_embedding[1]))

[[ 0.32395157 -1.41869982  1.77513966 ...,  0.9963549   0.44035243
   0.79483814]
 [ 0.02164781  0.91206423 -0.09118133 ..., -1.56139498  0.15066629
  -0.85542388]
 [ 0.07311916  0.04990149 -0.05353269 ...,  0.02184222  0.07796076
  -0.021182  ]
 [ 0.04701679 -0.00843652 -0.01348642 ...,  0.01990544  0.06836856
  -0.00276519]
 [ 0.15583396  0.05792532  0.1083082  ...,  0.05198137  0.03322491
  -0.06788541]]
411722
256


In [30]:
sr_que2id=open('../data/sr_word2id.pkl','rb')
sr_que2id_2=pk.load(sr_que2id)#读取pkl内容
print(type(sr_que2id_2))
print(sr_que2id_2.shape)
print(sr_que2id_2[:5])
print(sr_que2id_2[411710:411722])

<class 'pandas.core.series.Series'>
(411722,)
2    </s>
3     w11
4     w54
5      w6
6    w111
dtype: object
411712    w1136515
411713    w1136999
411714    w1137196
411715      w59241
411716      w73801
411717      w76782
411718     w123523
411719     w125081
411720     w125663
411721     w133825
0            <PAD>
1            <EOS>
dtype: object


### char2id 
利用上面得到的 sr_char2id，把所有问题的字转为对应的id, 存储为
- data/ch_train_title.npy
- data/ch_train_content.npy
- data/ch_eval_title.npy
- data/ch_eval_content.npy

In [39]:
question_train_set_r = open('../raw_data/question_eval_set.txt',"r")
question_train_lines = question_train_set_r.readlines()
print(len(question_train_lines))
question_train_lines[0]

217360


'6215603645409872328\tc924,c531,c102,c284,c188,c104,c98,c107,c11,c117,c118,c91,c92,c960,c3225,c518,c419,c184,c622,c518,c419,c665,c936,c572,c1321,c11,c275,c2125,c184\tw1340,w1341,w55,w1344,w58,w6,w24178,w26959,w471,w111,w642,w471,w3228,w20104,w19234,w6,w18505,w111\tc1128,c529,c636,c572,c1321,c139,c540,c223,c510,c24,c473,c153,c131,c17,c322,c674,c40,c924,c531,c2,c398,c835,c57,c652,c101,c96,c97,c97,c98,c99,c100,c101,c373,c275,c1818,c2637,c38\tw4094,w1618,w20104,w19234,w1097,w1005,w4228,w2160,w72,w11,w215,w1067,w1340,w1341,w848,w1009,w686,w54,w53,w54,w139426,w23\n'

In [45]:
df_trains = pd.read_csv('../raw_data/question_eval_set.txt', sep='\t', usecols=[0, 2, 4],
                            iterator = True, 
                            names=['question_id', 'char_title', 'char_content'], dtype={'question_id': object})
    #数据太大，只读取前100W行（约三分之一）
df_train = df_trains.get_chunk(10)
df_train

,question_id,char_title,char_content
0,6215603645409872328,"w1340,w1341,w55,w1344,w58,w6,w24178,w26959,w47...","w4094,w1618,w20104,w19234,w1097,w1005,w4228,w2..."
1,6649324930261961840,"w40132,w1357,w1556,w1380,w2464,w33,w16791,w109...",NaN
2,-4251899610700378615,"w53,w54,w1779,w54,w1309,w54,w369,w949,w65587,w...",NaN
3,6213817087034420233,"w5083,w12537,w10427,w29724,w6,w2566,w11,w18476...","w2550,w24,w239,w98,w19456,w11,w108710,w3483,w2..."
4,-8930652370334418373,"w33792,w21,w83,w6,w21542,w21,w140670,w25,w1110...",NaN
5,-2893445476547593888,"w381,w68713,w5324,w662,w6,w4852,w54,w251,w54,w...",NaN
6,2614833994648160978,"w30288,w10523,w18688,w140728,w346,w1515,w298,w...","w18688,w48991,w21361,w140729,w132351,w140730,w..."
7,1572988006266661060,"w41,w2185,w90296,w6,w10129,w1619,w19351,w1621,...",NaN
8,-3736249911643942320,"w305,w8568,w69,w109,w11978,w6,w2292,w0,w14288,...","w3587,w1432,w6,w25,w2292,w0,w1613,w8568,w2558,..."
9,-976507019126932319,"w4055,w6,w54,w138,w54,w510,w54,w632,w1038,w275...","w5800,w25,w106739,w349,w8768,w278,w6,w11,w1924..."


In [41]:
ch_train_title = np.load('../data/wd_eval_title.npy')
ch_train_title[0:5]

array([ list([2337, 1469, 70, 49242, 69, 5, 146157, 1037, 54, 6, 133, 54, 403, 3691, 5927, 5, 678, 6]),
       list([26005, 129, 466, 63, 8088, 33, 21039, 147, 112377, 5, 281, 12, 5351, 40, 6]),
       list([3425, 4, 4047, 4, 53553, 4, 2076, 1178, 118189, 91, 85, 103, 28, 404, 251, 6]),
       list([329, 1544, 640, 50275, 5, 281, 3, 298, 27, 1718, 57316, 6]),
       list([9560, 26, 1057, 5, 18077, 26, 188558, 9, 27, 6])], dtype=object)

### question_and_topic_2id
把问题和话题转为id形式，保存在 data/sr_question2id.pkl 和 data/sr_id2question.pkl 中

In [58]:
import pickle as pk

# rb 以二进制格式打开一个文件用于只读
f=open('../data/sr_question2id.pkl','rb')
inf=pk.load(f)#读取pkl内容
print(type(inf))
print(inf.shape)
print(inf[0:5])

<class 'pandas.core.series.Series'>
(2999967,)
6555699376639805223     0
2887834264226772863     1
-2687466858632038806    2
-5698296155734268       3
-6719100304248915192    4
dtype: int64


In [60]:
import pickle as pk

# rb 以二进制格式打开一个文件用于只读
f=open('../data/sr_topic2id.pkl','rb')
inf=pk.load(f)#读取pkl内容
print(type(inf))
print(inf.shape)
print(inf[0:5])

<class 'pandas.core.series.Series'>
(1999,)
7476760589625268543     0
4697014490911193675     1
-4653836020042332281    2
-8175048003539471998    3
-8377411942628634656    4
dtype: int64


In [61]:
y_tr = np.load('../data/y_tr.npy')
print(y_tr[0:5])
print(len(y_tr))
print(len(y_tr[1]))

[list([744, 36]) list([1385]) list([326]) list([1364, 98])
 list([85, 123, 1724, 202, 514])]
2999952
1


### creat_batch_data
把所有的数据按照 batch_size(128) 进行打包，固定seed，随机取 10 万样本作为验证集。每个batch存储为一个 npz 文件，包括 X, y 两部分。 这里所有的序列都进行了截断，长度不足的用0进行padding到固定长度。
保存位置：
- wd_train_path = '../data/wd-data/data_train/'
- wd_valid_path = '../data/wd-data/data_valid/'
- wd_test_path = '../data/wd-data/data_test/'
- ch_train_path = '../data/ch-data/data_train/'
- ch_valid_path = '../data/ch-data/data_valid/'
- ch_test_path = '../data/ch-data/data_test/'

In [13]:
batch_data = np.load('../data/wd-data/data_test/0.npy')
print(type(batch_data))
print(batch_data.shape)
batch_data[0]

<class 'numpy.ndarray'>
(128, 180)


array([  2337,   1469,     70,  49242,     69,      5, 146157,   1037,
           54,      6,    133,     54,    403,   3691,   5927,      5,
          678,      6,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,   3856,    266,
         3691,   5927,    147,    148,    227,    993,     11,      3,
           94,    893,   2337,   1469,     38,     30,     65,      4,
         3425,      4,  81526,      7,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [8]:
batch_data = np.load('../data/wd-data/data_train/0.npz')
print(type(batch_data))
x = batch_data['X']
y = batch_data['y']
print(x.shape)
print(y.shape)
print(x[0:1])
print(y[0:1])

<class 'numpy.lib.npyio.NpzFile'>
(128, 180)
(128,)
[[ 27905    153    369  11501      5  16182   5728      6      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
      34   4839    161      5     55  45050  33199      3   3792    457
       5   1587    160      3    115  19111   1547   1663  18614      3
     228     29   6789     24   4665   4573    369  16182     29     11
       6      4   7705    318      5   5530     75   9144      5   1795
     475    386    160     11   1331      3  32794      3  96134      7
     175    506    369   9212   4906   5411    388      5    683    371
   16182     75  21434    187  10618  14679  10618    184  13420      6
      51     85     13  18140      6      4    110   2281   1861     75
      13     34    128    129    261   4522    548  22001   2870      4
     433      5   1029   1519    659      4   1520    261      5    590
    1029    